In [1]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.7/239.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB

In [2]:
pip install datasets

In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
import csv
import warnings
from typing import Any, Dict, Tuple
import re
from datasets import load_dataset
import os
from sklearn.metrics import classification_report


from transformers import AutoTokenizer, pipeline


warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

In [4]:
#load the data for all languages
data = load_dataset("Eloquent/HalluciGen-Translation", "trial")

print(data.keys())

data_de_en = data['test_detection_de_en'].to_pandas()
data_en_de = data['test_detection_en_de'].to_pandas()
data_fr_en = data['test_detection_fr_en'].to_pandas()
data_en_fr = data['test_detection_en_fr'].to_pandas()

display(data_de_en.head())
display(data_en_de.head())

Some datasets params were ignored: ['sep']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.


Generating trial_de_en split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_en_de split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_fr_en split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_en_fr split:   0%|          | 0/10 [00:00<?, ? examples/s]

dict_keys(['trial_de_en', 'trial_en_de', 'trial_fr_en', 'trial_en_fr'])


KeyError: 'test_detection_de_en'

# Dataset

In [ ]:
#load the data for all languages
data = load_dataset("Eloquent/HalluciGen-Translation", "test_detection")

print(data.keys())

data_de_en = data['test_detection_de_en'].to_pandas()
data_en_de = data['test_detection_en_de'].to_pandas()
data_fr_en = data['test_detection_fr_en'].to_pandas()
data_en_fr = data['test_detection_en_fr'].to_pandas()

display(data_de_en.head())
display(data_en_de.head())

Some datasets params were ignored: ['sep']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.


Generating test_detection_de_en split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_detection_en_de split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_detection_fr_en split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_detection_en_fr split:   0%|          | 0/100 [00:00<?, ? examples/s]

dict_keys(['test_detection_de_en', 'test_detection_en_de', 'test_detection_fr_en', 'test_detection_en_fr'])


,id,langpair,source,hyp1,hyp2
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, offering luxury guesthouses has ...","In developed countries, offering luxury guesth..."
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","This is common practice in the UK, but Scottis...","This is common practice elsewhere in the UK, b..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,"“Based on this fossil, we can say that the sep...","“Based on this fossil, we can say that the sep..."
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...","We will never overtake a car, build a well in ...","We will never overtake a car, build a well in ..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...","An Avenger class anti-mining ship, the ship wa...","An Avenger class anti-mining ship, the ship wa..."


,id,langpair,source,hyp1,hyp2
0,0,en-de,He graduated from the College of Arts & Scienc...,Er absolvierte 1950 das College of Arts & Scie...,Er absolvierte 1950 das College of Arts & Scie...
1,1,en-de,"Most are independent nations, or associated wi...",Die meisten der kleineren Inseln sind unabhäng...,"Die meisten sind unabhängige Nationen, oder mi..."
2,2,en-de,But there are a lot of things about birds that...,"Aber es gibt viele Dinge über Vögel, die immer...","Aber es gibt viele Dinge über Vögel, die wie e..."
3,3,en-de,The police had trouble using their speed radar...,"Die Polizei hatte Schwierigkeiten, ihre Geschw...",Die neuseeländische Polizei hatte Schwierigkei...
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t...","""Das Plateau von Giza oder """"Nekropolis von Gi...","""Das Plateau von Giza oder """"Nekropolis von Gi..."


In [ ]:

#load the trial data for all language pairs
trial_ds = load_dataset("Eloquent/HalluciGen-Translation", name="trial")
print(trial_ds.keys())

Some datasets params were ignored: ['sep']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.


Generating trial_de_en split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_en_de split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_fr_en split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_en_fr split:   0%|          | 0/10 [00:00<?, ? examples/s]

dict_keys(['trial_de_en', 'trial_en_de', 'trial_fr_en', 'trial_en_fr'])


In [ ]:


trial_ds_de_en = trial_ds['trial_de_en'].to_pandas()
trial_ds_en_de = trial_ds['trial_en_de'].to_pandas()
trial_ds_fr_en = trial_ds['trial_fr_en'].to_pandas()
trial_ds_en_fr = trial_ds['trial_en_fr'].to_pandas()

display(trial_ds_de_en.head())
display(trial_ds_en_de.head())



,id,langpair,source,hyp1,hyp2,type,label
0,0,de-en,In der Phase marschieren Wanderameisen bei Nac...,"In the phase, wandering ants march at night an...","In the nomadic phase, wandering ants march at ...",addition,hyp2
1,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1
2,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1
3,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1
4,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2


,id,langpair,source,hyp1,hyp2,type,label
0,0,en-de,The days in the summer can lead to problems ge...,"Die Tage im Sommer können zu Problemen führen,...",Die sehr langen Tage im Sommer können zu Probl...,addition,hyp2
1,1,en-de,The vertical clearance under the bridge is 15 ...,Der vertikale Abstand unter der Brücke beträgt...,Der vertikale Abstand unter der Brücke beträgt...,date,hyp1
2,2,en-de,William Llewelyn Williams known as Llewelyn Wi...,"William Williams, bekannt als Llewelyn William...","Williajm Llewelyn Williams, bekannt als Llewel...",named-entity,hyp2
3,3,en-de,The single was released on Radio Airplay in It...,Die Single wurde am 12. Oktober 212 in Italien...,Die Single wurde am 12. Oktober 2012 in Italie...,number,hyp1
4,4,en-de,Water is spilling over the levee in a section ...,Wasser fließt über den Deich in einem Abschnit...,Wasser fließt über die Leibe in einem Abschnit...,natural,hyp2


## Load Model

In [ ]:
#load model Llama3
model = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"label": ""}

    # prompt for translation generation task
    user_prompt = f'''You are a researcher investigating a new phenomenon. You have gathered data (source) and formulated
    two competing hypotheses (Hyp1 and Hyp2) to explain it.
    Identify the hypothesis that contradicts the information provided in the given source.
    Provide the result in the following format: {answer_format}.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''

    messages = [
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
import ast
def extract_json_data(result_text):
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    result_text = result_text.lower()
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    return result_text

### Test

In [ ]:
source = trial_ds_de_en.iloc[0]["source"]
hyp1 = trial_ds_de_en.iloc[0]["hyp1"]
hyp2 = trial_ds_de_en.iloc[0]["hyp2"]
hallucination = detect_hallucination(source, hyp1, hyp2, "English")
print(hallucination)
json_data = extract_json_data(hallucination)
print(json_data)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'Hyp2'}

Explanation: The source states "In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast." which translates to "In the phase, wandering ants march at night and rest during the day." This information directly contradicts Hyp2, which also states that wandering ants march at night and rest during the day.
{'label': 'Hyp2'}

Explanation: The source states "In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast." which translates to "In the phase, wandering ants march at night and rest during the day." This information directly contradicts Hyp2, which also states that wandering ants march at night and rest during the day.
{'label': 'Hyp2'}


In [ ]:
trial_ds_de_en.iloc[0]["label"]


'hyp2'

# Test with Trial dataset

In [ ]:
#save results
def save_trial_results(df, csv_filename, translation_language):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']
            hyp1 = row["hyp1"]
            hyp2 = row["hyp2"]

            try:
                # Generate translation
                result = detect_hallucination(source, hyp1, hyp2, translation_language)
                extracted_hallu = extract_json_data(result)
                hallu_label = extracted_hallu["label"]
                match = re.search(r'hyp2|hyp1', hallu_label)


                # Update DataFrame with new values
                chunk.at[index, "pred_label"] = match.group()
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        #display(df)
    return skipped_rows, df

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_results_llama3_de_en_narjes_prompt.csv"
skipped_rows = save_trial_results(trial_ds_de_en, csv_filename, "English")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'Hyp2'}

Explanation: The source text mentions "Phase marschieren Wanderameisen" which translates to "In the phase, wandering ants march". This implies that the ants are marching during the phase, and the day/night information is not relevant to the phase itself. Therefore, Hyp2, which mentions "nomadic phase", contradicts the information provided in the source.
result_text  {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'Hyp1'}

The source states that the performance occurred on August 5, not December 5, so Hyp1 contradicts the information provided in the source.
result_text  {'label': 'hyp1'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

detected hallucination:  {'label': 'Hyp2'}

Explanation: The source states that the distance from the earthquake center to Butte is approximately 65 km (40 miles), which is a reasonable and plausible value. Hyp2, on the other hand, suggests a distance of approximately 65000 metres, which is equivalent to about 65,000 kilometers, which is an absurdly large distance. This contradicts the information provided in the source, making Hyp2 the hypothesis that contradicts the data.
result_text  {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'Hyp2'}
result_text  {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  The correct result is:

{'label': 'None','message': 'Both hypotheses (Hyp1 and Hyp2) are identical and do not contradict the source.'}

The source states that container glass has a higher content of magnesium oxide and sodium oxide compared to flat glass, and a lower content of silica, calcium oxide, and aluminum oxide. Both Hyp1 and Hyp2 accurately reflect this information, making them consistent with the source and not contradicting each other.
result_text  {'label': 'none','message': 'both hypotheses (hyp1 and hyp2) are identical and do not contradict the source.'}
Skipping row 8 due to error in data extraction.
detected hallucination:  {'label': 'Hyp2'}

Hypothesis 2 contradicts the information provided in the source, as it states that Kathy's wife is Pete Beale, which is not supported by the source. The source explicitly mentions that Kathy is married to Pete Beale, not that she is married to a woman named Pete Beale.
result_text  {'label': 'hyp2'}


## Generate dataset


In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_de_en_new_prompt.csv"
skipped_rows, df = save_trial_results(data_de_en, csv_filename, "English")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'Hyp2'}

The source text does not mention developed countries, it simply says "Ländern" which translates to "countries" in general. Therefore, Hyp2, which specifically mentions developed countries, contradicts the information provided in the source.
result_text  {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'Hyp2'}

Explanation: Hyp2 states that the practice is common elsewhere in the UK, which contradicts the source, which specifically mentions that it is common in Britain, implying that it is a nationwide practice, not limited to England or Wales.
result_text  {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

detected hallucination:  {'label': 'hyp2'}

The source states that the Ponoru is a Nebenfluss (which means "side stream" or "tributary" in German), implying that it flows into the Horezu River. Hyp2 states that the Ponoru is a tributary of the Horezu, which is consistent with the information provided in the source. On the other hand, Hyp1 states that the Ponoru is a distributary of the Horezu, which means it would flow out of the Horezu River, contradicting the information in the source.
result_text  {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'Hyp2'}

Hypothesis 2 contradicts the information provided in the source, which states that Alex Bernstein is a "Member of the Mediendynastie von Granada" (member of the Granada media dynasty).
result_text  {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'Hyp2'}

Hypothesis 2 ("In the meantime, the Air Force sold the SR71, now A-12, which we were working on with the CIA.") contradicts the information provided in the source, which states that the Air Force "kaufte" (bought) the SR71, now A-12.
result_text  {'label': 'hyp2'}
detected hallucination:  {'label': 'Hyp2'}

Hypothesis 2 states that Sher Ahmed Akhundzada is a tribal follower, which contradicts the information provided in the source, which clearly states that Sher Mohammed Akhundzada (also known as Sher Ahmed Akhundzada) is a tribal leader.
result_text  {'label': 'hyp2'}


#English-German

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_en_de_new_prompt.csv"
skipped_rows, df = save_trial_results(data_en_de, csv_filename, "German")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'Hyp1'}

Hyp1: Er absolvierte 1950 das College of Arts & Sciences der University of Virginia und war ein bedeutender Spender dieser Institution.

The source does not mention the University of Virginia, whereas Hyp1 specifically mentions it.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'hyp2 does not contradict the source, hyp1 contradicts the source'}

The source states "Most are independent nations, or associated with France, and known as luxury beach resorts." Hyp1 is a direct translation of this sentence, so it does not contradict the source.

Hyp2 is identical to Hyp1, so it also does not contradict the source.

However, if we look closely, Hyp1 is actually a repetition of the source, whereas Hyp2 is a more general statement that encompasses the information in the source.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

detected hallucination:  Based on the source, I would say that Hyp1 contradicts the information provided.

The source states: "The amazing thing about this is this is, I mean, obviously, these are old games, but the system just sees what you see, the pixels, and it has the controls and it has the score, and it's learned to play all of these games, same program."

Hyp1 appears to be a direct translation of this statement, and it does not contradict the source.

On the other hand, Hyp2 is identical to Hyp1, and it does not provide any new information or contradict the source.

Therefore, the result is:

{'label': 'Hyp1'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'Hyp2'}

The source text mentions "it has the controls and it has the score", which implies that the system has learned to play the games with the controls and score provided. However, Hyp2 states "er hat die Kontrolle", which translates to "he has the control", using the masculine pronoun "er", implying that a human is in control, not the system. This contradicts the information provided in the source, which suggests that the system is playing the games independently.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'Hyp2'}

Explanation:
The source text contains the phrase "I had to give it up", which implies that the person had to stop doing something (in this case, having a job) because of an inability to leave the house. Hyp1 is a direct translation of the source text, whereas Hyp2 changes "Job" to "es" (which means "it" in German), which doesn't match the original text.
detected hallucination:  {'label': 'Hyp2'}

Hyp2 is the hypothesis that contradicts the information provided in the source. The source states that the system "has learned to play all of these games, same program", which implies that the system has learned to play different games using the same program. Hyp2 does not contradict this statement.

On the other hand, Hyp1 does not contradict the source, as it also states that the system has learned to play all of these games using the same program.

Note: Since both Hyp1 and Hyp2 are identical, it is not possible to identify a contradiction based o

In [ ]:
skipped_rows

[5, 22, 34, 94]

# English-French

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_en_fr_new_prompt.csv"
skipped_rows, df = save_trial_results(data_en_fr, csv_filename, "French")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'Hyp2'}

Hypothesis 2 states that Bobek was born in Zagreb, the Croatian capital, which contradicts the information provided in the source that he was born in the Croatian capital, which is implied to be Belgrade (not Zagreb).


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'None'}

Both Hyp1 and Hyp2 are identical, they both state that using GPS navigation apps is the easiest and most convenient way to navigate when out of your home country. Therefore, neither of them contradicts the information provided in the source.
Skipping row 1 due to error in data extraction.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

detected hallucination:  {'label': 'Hyp2'}

Explanation: Hyp2 contradicts the information provided in the source because it states that Flavia Alejandra Gleske Fajin was born on May 11, 1978, whereas the source states that she was born on May 15, 1978.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'hyp2'}

Explanation: 

The source states that the person was elected four times for Al Smith in 1928, for Franklin D. Roosevelt, and for Harry S. Truman in 1948. 

Hyp1 is consistent with this information, stating that Massachusetts voted for Al Smith in 1928, for Franklin D. Roosevelt four times in the 1930s and 1940s, and for Harry S. Truman in 1948.

Hyp2, on the other hand, contains errors. It states that the person was elected four times "à travers" Al Smith in 14928, which is incorrect since the year 14928 does not exist. Additionally, it does not mention the election for Franklin D. Roosevelt, which is mentioned in the source. Therefore, Hyp2 contradicts the information provided in the source.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'Hyp2'}

Hyp2: Il était orateur invité de l'ICM à Toronto en 1924, en 1932 à Zurich et en 1936 à Oslo.

The source states that the events occurred in 1932 in Zurich and in 1936 in Oslo, but Hyp2 does not mention the year for the Zurich event, contradicting the information provided in the source.
detected hallucination:  {'label': 'hyp2'}

Hyp2 contradicts the information provided in the source, which states that the acquisition of the remainder of E-Plus occurred on March 14, not March 124.


In [ ]:
skipped_rows

[1, 2, 38, 43, 44, 47, 48, 54, 58, 59, 75]

## French-English

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_fr_en_new_prompt.csv"
skipped_rows, df = save_trial_results(data_fr_en, csv_filename, "English")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'Hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'Hyp2'}

Hyp2: Pronunciation is relatively easy in Italian, as most words are pronounced exactly as they are written

The source states: "est relativement facile en italien, car la plupart des mots sont prononcés exactement comme ils sont écrits"

The key phrase in the source is "sont prononcés exactement comme ils sont écrits", which means "are pronounced exactly as they are written". This implies that the ease of pronunciation lies in the words themselves, not in the pronunciation itself. Therefore, Hyp2, which states that "Pronunciation is relatively easy in Italian", contradicts the information provided in the source.

Hyp1, on the other hand, correctly states that "is relatively easy in Italian, as most words are pronounced exactly as they are written", which aligns with the information provided in the source.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

detected hallucination:  {'label': 'hyp1'}

The source states that the Arieşul Mare River is an affluent (which means a tributary or a stream that flows into a larger river) of the Vâlcea River, not a distributary (which is a stream that flows away from a larger river). Therefore, hypothesis 1 (Hyp1) contradicts the information provided in the source.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'hyp1'}

The source states that Born was born in Germany to Jewish parents, the son of Max Born and the scientist Hedwig Ehrenberg. Hyp1 contradicts this information by stating that Born was the daughter of Max Born and Hedwig Ehrenberg.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'Hyp2'}

The source states that the Wallowa River is an affluent (a distributary) of the Lostine River, which means it flows into the Lostine River. Hyp2 contradicts this information by stating that the Wallowa River is a tributary (which flows out of) the Lostine River.
detected hallucination:  {'label': 'Hyp2'}

The source states that La rivière Vâlcea est un affluent de la rivière Arieşul Mare en Roumanie, which translates to "The Vălcea river is a tributary of the River Arieşul Mare in Romania." This information contradicts Hyp2, which states that the Vâlcea River is a distributary of the Arieşul Mare River.


In [ ]:
skipped_rows

[3, 4, 15, 23, 26, 29, 32, 75, 76, 77, 82, 86]

In [ ]:
#########

## Convert to comma-separated csv

In [ ]:
import pandas as pd
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_de_en_new_prompt_final.csv"


df = pd.read_csv(csv_filename, sep=";")



In [ ]:
df.head()

,id,pred_label
0,0,hyp2
1,1,hyp2
2,2,hyp2
3,3,hyp2
4,4,hyp1


In [ ]:
df.to_csv(csv_filename, mode='w', header=True, index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_en_fr_new_prompt_final.csv"


df = pd.read_csv(csv_filename, sep=";")
df.head()


,id,pred_label
0,0,hyp2
1,1,NaN
2,2,NaN
3,3,hyp2
4,4,hyp1


In [ ]:
df.to_csv(csv_filename, mode='w', header=True, index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_fr_en_new_prompt_final.csv"


df = pd.read_csv(csv_filename, sep=";")
df.head()


,id,pred_label
0,0,hyp2
1,1,hyp2
2,2,hyp2
3,3,NaN
4,4,NaN


In [ ]:
df.to_csv(csv_filename, mode='w', header=True, index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_en_de_new_prompt_final.csv"


df = pd.read_csv(csv_filename, sep= ";")
df.head()


,id,pred_label
0,0,hyp1
1,1,hyp2
2,2,hyp1
3,3,hyp2
4,4,hyp2


In [ ]:
df.to_csv(csv_filename, mode='w', header=True, index=False)

In [ ]:
### Detect number of blank rows
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_de_en_new_prompt_final.csv"


df = pd.read_csv(csv_filename)

In [ ]:
df.head()

,id,pred_label
0,0,hyp2
1,1,hyp2
2,2,hyp2
3,3,hyp2
4,4,hyp1


In [ ]:
# Using isna() to check for NaN values
count_nan = df['pred_label'].isna().sum()

# Using an additional condition to check for empty strings
count_empty_or_nan = df['pred_label'].isna().sum() + (df['pred_label'] == '').sum()

print(f'Number of rows with NaN in "label": {count_nan}')
print(f'Number of rows with empty or NaN in "label": {count_empty_or_nan}')

Number of rows with NaN in "label": 4
Number of rows with empty or NaN in "label": 4
